In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import numpy as np
import matplotlib.pylab as plt
import os
from datetime import datetime

In [ ]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

In [ ]:
np.unique(train_labels)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

In [ ]:
# Plain text name in alphabetical order. https://www.cs.toronto.edu/~kriz/cifar.html
CLASS_NAMES = ['airplane', 'automobile', 'bird', 'cat', 
               'deer','dog', 'frog', 'horse', 'ship', 'truck']

Let's use half of test data as the validation data.

In [ ]:
validation_dataset = tf.data.Dataset.from_tensor_slices((test_images[:500], test_labels[:500]))
test_dataset = tf.data.Dataset.from_tensor_slices((test_images[500:], test_labels[500:]))

Now the datasets are built for validation and test data. Next, we will keep all training data for training.

In [ ]:
# Create an instance of dataset from raw numpy images and labels.
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))

In [ ]:
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset#transformations_2
train_dataset_size = len(list(train_dataset.as_numpy_iterator()))
print('Training data sample size: ', train_dataset_size)

validation_dataset_size = len(list(validation_dataset.as_numpy_iterator()))
print('Validation data sample size: ', validation_dataset_size)

test_dataset_size = len(list(test_dataset.as_numpy_iterator()))
print('Test data sample size: ', test_dataset_size)

Training data sample size:  50000
Validation data sample size:  500
Test data sample size:  9500


In [ ]:
TRAIN_BATCH_SIZE = 128
train_dataset = train_dataset.shuffle(50000).batch(TRAIN_BATCH_SIZE, drop_remainder=True)

In [ ]:
validation_dataset = validation_dataset.batch(validation_dataset_size)
test_dataset = test_dataset.batch(test_dataset_size)

In [ ]:
STEPS_PER_EPOCH = train_dataset_size // TRAIN_BATCH_SIZE
VALIDATION_STEPS = 1 #validation data // validation batch size

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', 
      kernel_initializer='glorot_uniform', padding='same', input_shape = (32,32,3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu', 
      kernel_initializer='glorot_uniform', padding='same'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu', kernel_initializer='glorot_uniform'),
    tf.keras.layers.Dense(10, activation='softmax', name = 'custom_class')
])
model.build([None, 32, 32, 3])

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4096)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               1048832   
_________________________________________________________________
custom_class (Dense)         (None, 10)               

In [ ]:
model.compile(
          loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
          optimizer='adam',
          metrics=['accuracy'])

Let's define some alias for file path to save model checkpoints.

In [ ]:
MODEL_NAME = 'myCIFAR10-{}'.format(datetime.now().strftime("%Y%m%d-%H%M%S"))
print(MODEL_NAME)

myCIFAR10-20210123-212138


In [ ]:
checkpoint_dir = './' + MODEL_NAME
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt-{epoch}")
print(checkpoint_prefix)

./myCIFAR10-20210123-212138/ckpt-{epoch}


See https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint for details of function signature for `ModelCheckpoint`.

In [ ]:
myCheckPoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    monitor='val_accuracy',
    mode='max')
myEarlyStop = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', 
                                               patience=4)

In [ ]:
myCallbacks = [
    myCheckPoint,
    myEarlyStop
]

In [ ]:
hist = model.fit(
    train_dataset,
    epochs=20,
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_data=validation_dataset,
    validation_steps=VALIDATION_STEPS,
    callbacks=myCallbacks).history

Epoch 1/20
250/250 [==============================] - 5s 11ms/step - loss: 1.7964 - accuracy: 0.3526 - val_loss: 1.2781 - val_accuracy: 0.5540
INFO:tensorflow:Assets written to: ./myCIFAR10-20210123-212138/ckpt-1/assets
Epoch 2/20
250/250 [==============================] - 4s 13ms/step - loss: 1.1889 - accuracy: 0.5835 - val_loss: 1.0760 - val_accuracy: 0.6200
INFO:tensorflow:Assets written to: ./myCIFAR10-20210123-212138/ckpt-2/assets
Epoch 3/20
250/250 [==============================] - 4s 13ms/step - loss: 1.0184 - accuracy: 0.6423 - val_loss: 0.9688 - val_accuracy: 0.6420
INFO:tensorflow:Assets written to: ./myCIFAR10-20210123-212138/ckpt-3/assets
Epoch 4/20
250/250 [==============================] - 4s 13ms/step - loss: 0.9197 - accuracy: 0.6804 - val_loss: 0.9514 - val_accuracy: 0.6640
INFO:tensorflow:Assets written to: ./myCIFAR10-20210123-212138/ckpt-4/assets
Epoch 5/20
250/250 [==============================] - 4s 13ms/step - loss: 0.8542 - accuracy: 0.7044 - val_loss: 0.9280 

In [ ]:
type(hist)

dict

In [ ]:
hist['val_accuracy']

[0.47200000286102295,
 0.5680000185966492,
 0.6000000238418579,
 0.5899999737739563,
 0.6119999885559082,
 0.6019999980926514,
 0.6100000143051147,
 0.6380000114440918,
 0.6100000143051147,
 0.5699999928474426,
 0.5619999766349792,
 0.5960000157356262]

In [ ]:
max_value = max(hist['val_accuracy'])
max_index = hist['val_accuracy'].index(max_value)
print('Best epoch: ', max_index + 1)

Best epoch:  8


Epoch 7 yielded model with highest accuracy. Now take a look at the checkpoint directory:

In [ ]:
!ls -lrt ./cifar10_training_checkpoints

total 48
drwxr-xr-x 4 root root 4096 Jan 14 01:56 ckpt_1
drwxr-xr-x 4 root root 4096 Jan 14 01:56 ckpt_2
drwxr-xr-x 4 root root 4096 Jan 14 01:56 ckpt_3
drwxr-xr-x 4 root root 4096 Jan 14 01:57 ckpt_4
drwxr-xr-x 4 root root 4096 Jan 14 01:57 ckpt_5
drwxr-xr-x 4 root root 4096 Jan 14 01:57 ckpt_6
drwxr-xr-x 4 root root 4096 Jan 14 01:57 ckpt_7
drwxr-xr-x 4 root root 4096 Jan 14 01:57 ckpt_8
drwxr-xr-x 4 root root 4096 Jan 14 01:57 ckpt_9
drwxr-xr-x 4 root root 4096 Jan 14 01:57 ckpt_10
drwxr-xr-x 4 root root 4096 Jan 14 01:57 ckpt_11
drwxr-xr-x 4 root root 4096 Jan 14 01:57 ckpt_12


In [ ]:
!ls -lrt ./cifar10_training_checkpoints/ckpt_7

total 136
drwxr-xr-x 2 root root   4096 Jan 14 01:57 variables
drwxr-xr-x 2 root root   4096 Jan 14 01:57 assets
-rw-r--r-- 1 root root 127273 Jan 14 01:57 saved_model.pb


In [ ]:
best_only_checkpoint_dir = './best_only_cifar10_training_checkpoints'
best_only_checkpoint_prefix = os.path.join(best_only_checkpoint_dir, "ckpt_{epoch}")
print(best_only_checkpoint_prefix)

./best_only_cifar10_training_checkpoints/ckpt_{epoch}


In [ ]:
bestCheckPoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=best_only_checkpoint_prefix
    ,monitor='val_accuracy'
    ,mode='auto'
    ,save_best_only=True)

In [ ]:
bestCallbacks = [
    bestCheckPoint
]

In [ ]:
best_hist = model.fit(
    train_dataset
    ,epochs=12
    ,steps_per_epoch=STEPS_PER_EPOCH
    ,validation_data=validation_dataset
    ,validation_steps=VALIDATION_STEPS
    ,callbacks=bestCallbacks).history

Epoch 1/12
250/250 [==============================] - 3s 10ms/step - loss: 0.3301 - accuracy: 0.8901 - val_loss: 2.0081 - val_accuracy: 0.6000
INFO:tensorflow:Assets written to: ./best_only_cifar10_training_checkpoints/ckpt_1/assets
Epoch 2/12
250/250 [==============================] - 3s 9ms/step - loss: 0.3254 - accuracy: 0.8950 - val_loss: 2.0864 - val_accuracy: 0.6000
Epoch 3/12
250/250 [==============================] - 3s 9ms/step - loss: 0.3404 - accuracy: 0.8919 - val_loss: 2.2601 - val_accuracy: 0.6020
INFO:tensorflow:Assets written to: ./best_only_cifar10_training_checkpoints/ckpt_3/assets
Epoch 4/12
250/250 [==============================] - 3s 9ms/step - loss: 0.3122 - accuracy: 0.9012 - val_loss: 2.2877 - val_accuracy: 0.5960
Epoch 5/12
250/250 [==============================] - 3s 9ms/step - loss: 0.2989 - accuracy: 0.9095 - val_loss: 2.3951 - val_accuracy: 0.6000
Epoch 6/12
250/250 [==============================] - 3s 9ms/step - loss: 0.2640 - accuracy: 0.9184 - val_los

Let's take a look at checkpoint directory where you set `set_best_only` to `True`:

In [ ]:
!ls -lrt ./best_only_cifar10_training_checkpoints

total 12
drwxr-xr-x 4 root root 4096 Jan 14 01:57 ckpt_1
drwxr-xr-x 4 root root 4096 Jan 14 01:58 ckpt_3
drwxr-xr-x 4 root root 4096 Jan 14 01:58 ckpt_9


Not all checkpoints are saved. This is because `save_best_only` option lets you save checkpoints after the first epoch only if there is an incremental improvement to the model metric of your choice. You can how many times the model was saved. This means the model performance in validation accuracy (`val_accuracy`) is best at the last time the model was saved. Therefore the last checkpoint saved is the best model.